In [219]:
# Import dependencies
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import sqlite3

from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

# Setting engine for SQLite connection
engine = create_engine("sqlite:///birthdata.sqlite", echo=False)


## Starting SQLite Connection

In [220]:
# Checking classes were made successfully: https://stackoverflow.com/questions/42946174/sqlalchemy-automap-not-generating-base-classes-table-name
engine = create_engine("sqlite:///birthdata.sqlite", echo=False)

# Declare a Base using `automap_base()`
Base = automap_base()

# Use the Base class to reflect the database tables
Base.prepare(engine, reflect=True)

# Print all of the classes mapped to the Base
print(Base.classes.keys())

# Create a session
session = Session(engine)


['clinic2010_county', 'clinic2010_state', 'clinic2015_county', 'clinic2015_state', 'county', 'national', 'outcomes']


### Naming Classes

In [221]:
# Assign the classes to variables
County2015 = Base.classes.clinic2015_county
State2015 = Base.classes.clinic2015_state
County2010 = Base.classes.clinic2010_county
State2010 = Base.classes.clinic2010_state
Outcomes = Base.classes.outcomes
County_births = Base.classes.county
State_births = Base.classes.national

## 1 Table Approach

### Selecting 1 Table for Modeling

In [103]:
# Switch out "selected" with the table of interest 
selected = County2010

# Display the row's columns and data in dictionary format
first_row = session.query(selected).first()
first_row.__dict__


{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x7fd6da710ca0>,
 'total_titleten': 1,
 'pp_client_tt': 0,
 'state': 'Alabama',
 'hospital_client': 0,
 'county': 'Autauga ',
 'fed_center_tt': 0,
 'other_clinic_client_tt': 0,
 'total_clinics': 2,
 'pp_client': 0,
 'dept_clinic_tt': 1,
 'fed_center': 1,
 'other_clinic_client': 0,
 'hospital_tt': 0,
 'index': 0,
 'dept_clinic': 1,
 'total_client_tt': 1120.0,
 'pp_tt': 0,
 'hospital': 0,
 'fed_client_tt': 0,
 'other_clinic_tt': 0,
 'pp': 0,
 'dept_clinic_client_tt': 1120,
 'total_client': 1280.0,
 'other_clinic': 0,
 'hospital_client_tt': 0,
 'fips': 1001,
 'fed_client': 160,
 'dept_clinic_client': 1120}

In [134]:
first_row = session.query(County_births).first()
first_row.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x7fd2d5391dc0>,
 'combined_fips_code': '1001',
 'state_fips_code': '01',
 'state': 'Alabama',
 'index': 0,
 'upper_confidence_limit': 55.2,
 'birth_rate': 48.4,
 'county_fips_code': '1',
 'county': 'Autauga',
 'year': 2003,
 'lower_confidence_limit': 42.1}

### Getting Query Results from 1 Table

In [4]:
# https://www.census.gov/data/tables/time-series/demo/popest/2010s-state-total.html#par_textimage_1574439295

import csv


In [115]:
url = "county_populations.csv"
state_populations = pd.read_csv(url)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf1 in position 2: invalid continuation byte

In [6]:
state_populations.head()

,state,2010_population,2015_population
0,United States,309321666,320635163
1,Alabama,4785437,4852347
2,Alaska,713910,737498
3,Arizona,6407172,6829676
4,Arkansas,2921964,2978048


In [7]:
state_populations.dtypes

state              object
2010_population     int64
2015_population     int64
dtype: object

In [8]:
state_expenditures = pd.read_csv("family_planning_expenditures.csv", header=0, names=["state", "MCH block grant", "Medicaid_spending", "SSBG and TANF", "Title10_spending", "state appropriations", "total_spending"])

In [9]:
state_expenditures.dtypes

state                   object
MCH block grant         object
Medicaid_spending        int64
SSBG and TANF           object
Title10_spending         int64
state appropriations    object
total_spending           int64
dtype: object

In [10]:
state_expenditures = state_expenditures[["state","Medicaid_spending", "Title10_spending","total_spending"]]


In [11]:
state_expenditures.head()

,state,Medicaid_spending,Title10_spending,total_spending
0,Alabama,58362,58362,69742
1,Alaska,2812,2812,10158
2,Arizona,52006,52006,57560
3,Arkansas,6899,6899,11397
4,California,438559,438559,454706


In [222]:
county_populations = pd.read_csv("county_populations.csv", encoding='latin-1')

In [223]:
county_populations.head(10)

,state,county,2010_population,2015_population
0,Iowa,Adair,7682,7145
1,Kentucky,Adair,18656,19162
2,Missouri,Adair,25607,25353
3,Oklahoma,Adair,22683,22259
4,Colorado,Adams,441603,490443
5,Idaho,Adams,3976,3891
6,Illinois,Adams,67103,66835
7,Indiana,Adams,34387,34945
8,Iowa,Adams,4029,3754
9,Mississippi,Adams,32297,31561


In [224]:
county_populations.dtypes

state              object
county             object
2010_population     int64
2015_population     int64
dtype: object

## Multi-Table Approach

### Getting Query Results from Several Tables

In [225]:
#states



# This cell can be used if you want to compare data from different tables.
# 1) Identify tables before the period and consider what is the most logical way to merge them.
# 2) Pull in the columns of interest from each table 
# 3) Write out their join, keeping in mind 1 to 1, many to 1, etc. 
# 4) Update column names to match, keeping the first column as the element they were joined on. 
# 5) Update index to that joined column, renaming as needed 
# Resource: https://www.kite.com/python/answers/how-to-join-multiple-tables-together-in-sqlalchemy-in-python#:~:text=Use%20Query.,sequence%20to%20tables%20to%20join.


join_query = session.query(State2015.total_clinics, State2015.total_titleten, State2015.pp,\
                           State2015.dept_clinic, State2015.dept_clinic_tt, State2015.hospital_tt,\
                           State2015.hospital, State2015.pp_tt, State2015.pp_client, \
                           State2015.total_client,State2015.total_client_tt,\
                           State_births.state_rate, State_births.year, State_births.state)\
                    .join(State_births, State_births.state == State2015.state)\
                    .filter(State_births.year=="2016").filter(State_births.age_group=="15-19 years")


state_df = pd.DataFrame(join_query, columns=["total_clinics", "total_title10", "total_pp", "dept_clinic", "dept_clinic_tt", "hospital_tt", "hospital", "pp_tt", "pp_client", "total_client", "total_client_tt", "birth_rate", "year", "state"])
#joined_df.set_index('state_name', inplace=True, )

# See code above to change data types if needed
# Drop NaN rows, which will mess with the ML
state_df = state_df.dropna()



In [226]:
state_df.head(55)

,total_clinics,total_title10,total_pp,dept_clinic,dept_clinic_tt,hospital_tt,hospital,pp_tt,pp_client,total_client,total_client_tt,birth_rate,year,state
0,187,83,2,77,75,0,1,0,260,97940,86180,28.4,2016,Alabama
1,157,5,5,16,1,0,0,3,3670,19570,5290,25.8,2016,Alaska
2,232,36,12,33,14,1,18,6,23930,84190,31820,23.6,2016,Arizona
3,163,92,2,92,90,0,0,2,4590,55430,51510,34.6,2016,Arkansas
4,1697,353,116,94,43,2,371,110,712470,1670600,1014320,17.0,2016,California
5,221,64,22,43,38,4,10,0,40930,118210,50280,17.8,2016,Colorado
6,92,20,17,0,0,1,9,13,54720,78740,46790,9.4,2016,Connecticut
7,41,38,3,22,21,0,0,3,4200,14490,13480,19.5,2016,Delaware
8,31,23,1,0,0,0,3,0,2900,36410,30750,24.0,2016,District of Columbia
9,430,142,23,123,112,0,3,5,48830,241450,149950,19.3,2016,Florida


In [227]:
# counties

join_query = session.query(County2010.total_clinics, County2010.total_titleten, County2010.pp,\
                           County2010.dept_clinic, County2010.hospital, County2010.total_client_tt,\
                           County2010.pp_client, County2010.dept_clinic_tt, County2010.pp_tt,\
                           County2010.total_client, County2010.hospital_client, County_births.birth_rate,\
                           County_births.year, County_births.state, County_births.county)\
                    .join(County_births, County_births.combined_fips_code == County2010.fips)\
                    .filter(County_births.year=="2011")


county_df = pd.DataFrame(join_query, columns=["total_clinics", "total_title10", "total_pp", "health_dept_clinics", "hospitals","title_10_clients","pp_clients", "dept_clinic_title10","pp_tt","total_clients","hospital_client","birth_rate", "year", "state", "county"])
#joined_df.set_index('state_name', inplace=True, )

# See code above to change data types if needed
# Drop NaN rows, which will mess with the ML
county_df = county_df.dropna()

In [229]:
county_df.loc[county_df["total_clinics"]==0]

,total_clinics,total_title10,total_pp,health_dept_clinics,hospitals,title_10_clients,pp_clients,dept_clinic_title10,pp_tt,total_clients,hospital_client,birth_rate,year,state,county
72,0,0,0,0,0,0.0,0,0,0,0.0,0,14.8,2011,Alaska,Denali
184,0,0,0,0,0,0.0,0,0,0,0.0,0,18.9,2011,California,Calaveras
242,0,0,0,0,0,0.0,0,0,0,0.0,0,41.6,2011,Colorado,Baca
247,0,0,0,0,0,0.0,0,0,0,0.0,0,15.8,2011,Colorado,Cheyenne
248,0,0,0,0,0,0.0,0,0,0,0.0,0,16.0,2011,Colorado,Clear Creek
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3085,0,0,0,0,0,0.0,0,0,0,0.0,0,4.9,2011,Wisconsin,Ozaukee
3102,0,0,0,0,0,0.0,0,0,0,0.0,0,12.8,2011,Wisconsin,Vernon
3116,0,0,0,0,0,0.0,0,0,0,0.0,0,43.3,2011,Wyoming,Converse
3123,0,0,0,0,0,0.0,0,0,0,0.0,0,28.1,2011,Wyoming,Lincoln


In [196]:
county_df["year"].unique()

array([2011])

In [197]:
county_df = county_df.merge(county_populations, how='left', on=["state","county"])

In [212]:
county_df = county_df.dropna()
county_df.shape

(2516, 31)

In [199]:
county_df.columns

Index(['total_clinics', 'total_title10', 'total_pp', 'health_dept_clinics',
       'hospitals', 'title_10_clients', 'pp_clients', 'dept_clinic_title10',
       'pp_tt', 'total_clients', 'hospital_client', 'birth_rate', 'year',
       'state', 'county', '2010_population', '2015_population'],
      dtype='object')

In [201]:
county_df["clinics_per_capita"] = county_df["total_clinics"]/county_df["2010_population"]
county_df["title10_clinics_per_capita"] = county_df["total_title10"]/county_df["2010_population"]
county_df["pp_per_capita"] = county_df["total_pp"]/county_df["2010_population"]
county_df["health_dept_per_capita"] = county_df['health_dept_clinics']/county_df["2010_population"]
county_df["hospitals_per_capita"] = county_df['hospitals']/county_df["2010_population"]
county_df["title_10_clients_per_capita"] = county_df["title_10_clients"]/county_df["2010_population"]
county_df["pp_clients_per_capita"] = county_df['pp_clients']/county_df["2010_population"]
county_df["dept_clinic_title10_per_capita"] = county_df['dept_clinic_title10']/county_df["2010_population"]
county_df["percent_clinics_tt_funded"] = county_df['total_title10']/county_df["total_clinics"]
county_df["percent_clients_at_pp"] = county_df['pp_clients']/county_df["total_clients"]
county_df["percent_clients_at_hospital"] = county_df['hospital_client']/county_df["total_clients"]
county_df["percent_clients_using_tt"] = county_df['title_10_clients']/county_df["total_clients"]

In [202]:
#binary encode existence of title ten funding for planned parenthood?

for index, row in county_df.iterrows():
    if row["total_pp"] == 0:
        county_df.loc[index, "binary_pp"]=0
    else:
        county_df.loc[index, "binary_pp"]=1
    if row["pp_tt"] == 0:
        county_df.loc[index, "binary_pp_tt"]=0
    else:
        county_df.loc[index, "binary_pp_tt"]=1

In [203]:
state_df = state_df.merge(state_populations, on="state", how="inner")

In [204]:
state_df = state_df.merge(state_expenditures, on="state", how="inner")

In [45]:
state_df.head()

,total_clinics,total_title10,total_pp,dept_clinic,dept_clinic_tt,hospital_tt,hospital,pp_tt,pp_client,total_client,total_client_tt,birth_rate,year,state,2010_population,2015_population,Medicaid_spending,Title10_spending,total_spending
0,187,83,2,77,75,0,1,0,260,97940,86180,28.4,2016,Alabama,4785437,4852347,58362,58362,69742
1,157,5,5,16,1,0,0,3,3670,19570,5290,25.8,2016,Alaska,713910,737498,2812,2812,10158
2,232,36,12,33,14,1,18,6,23930,84190,31820,23.6,2016,Arizona,6407172,6829676,52006,52006,57560
3,163,92,2,92,90,0,0,2,4590,55430,51510,34.6,2016,Arkansas,2921964,2978048,6899,6899,11397
4,1697,353,116,94,43,2,371,110,712470,1670600,1014320,17.0,2016,California,37319502,38918045,438559,438559,454706


In [46]:
state_df["clinics_per_capita"] = state_df["total_clinics"]/state_df["2015_population"]
state_df["title10_per_capita"] = state_df["total_title10"]/state_df["2015_population"]
state_df["pp_per_capita"] = state_df["total_pp"]/state_df["2015_population"]
state_df["health_dept_per_capita"] = state_df['dept_clinic']/state_df["2015_population"]
state_df["hospitals_per_capita"] = state_df['hospital']/state_df["2015_population"]
state_df["title_10_clients_per_capita"] = state_df["total_client_tt"]/state_df["2015_population"]
state_df["pp_clients_per_capita"] = state_df['pp_client']/state_df["2015_population"]
state_df["dept_clinic_title10_per_capita"] = state_df['dept_clinic_tt']/state_df["2015_population"]
state_df["percent_clinics_tt_funded"] = state_df['total_title10']/state_df["total_clinics"]
state_df["percent_pp_tt_funded"] = state_df['pp_tt']/state_df["total_pp"]
state_df["percent_clients_at_pp"] = state_df['pp_client']/state_df["total_client"]
state_df["Medicaid_spending_per_capita"] = state_df["Medicaid_spending"]/state_df["2015_population"]
state_df["Title10_spending_per_capita"] = state_df["Title10_spending"]/state_df["2015_population"]
state_df["total_spending_per_capita"] = state_df["total_spending"]/state_df["2015_population"]


In [142]:
county_df=county_df.dropna()

In [205]:
county_df.columns

Index(['total_clinics', 'total_title10', 'total_pp', 'health_dept_clinics',
       'hospitals', 'title_10_clients', 'pp_clients', 'dept_clinic_title10',
       'pp_tt', 'total_clients', 'hospital_client', 'birth_rate', 'year',
       'state', 'county', '2010_population', '2015_population',
       'clinics_per_capita', 'title10_clinics_per_capita', 'pp_per_capita',
       'health_dept_per_capita', 'hospitals_per_capita',
       'title_10_clients_per_capita', 'pp_clients_per_capita',
       'dept_clinic_title10_per_capita', 'percent_clinics_tt_funded',
       'percent_clients_at_pp', 'percent_clients_at_hospital',
       'percent_clients_using_tt', 'binary_pp', 'binary_pp_tt'],
      dtype='object')

## Prepping the Model Data

In [213]:
# Close the session from before
session.close()

# Assign the data to X and y
# Note: Sklearn requires a two-dimensional array of values so we use reshape to create this

X = county_df[['clinics_per_capita',
       'title10_clinics_per_capita', 'pp_per_capita', 'health_dept_per_capita',
       'hospitals_per_capita', 'title_10_clients_per_capita',
       'pp_clients_per_capita', 'dept_clinic_title10_per_capita',
       'percent_clinics_tt_funded', 'percent_clients_at_pp','binary_pp',
       'binary_pp_tt','percent_clients_at_hospital',
       'percent_clients_using_tt']]
y = county_df["birth_rate"]

feature_names = ['clinics_per_capita',
       'title10_clinics_per_capita', 'pp_per_capita', 'health_dept_per_capita',
       'hospitals_per_capita', 'title_10_clients_per_capita',
       'pp_clients_per_capita', 'dept_clinic_title10_per_capita',
       'percent_clinics_tt_funded', 'percent_clients_at_pp','binary_pp',
       'binary_pp_tt','percent_clients_at_hospital',
       'percent_clients_using_tt']

print("Shape: ", X.shape, y.shape)

Shape:  (2516, 14) (2516,)


In [214]:
# Use sklearn's `train_test_split` to split the data into training and testing
# Keep in mind that our group may want to use the model on an entirely different table as well. 

from sklearn.model_selection import train_test_split # may not apply to all models

### BEGIN SOLUTION 

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### END SOLUTION

## Linear Regression

In [215]:
# Create the model

### BEGIN SOLUTION

from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(max_depth=7, n_estimators=100, random_state=1)

### END SOLUTION

In [216]:
model.fit(X_train, y_train)

RandomForestRegressor(max_depth=7, random_state=1)

In [217]:
# Calculate the mean_squared_error and the r-squared value
# for the testing data

from sklearn.metrics import mean_squared_error, r2_score

### BEGIN SOLUTION

# Use our model to make predictions
predicted = model.predict(X_test)

# Score the predictions with mse and r2
mse = mean_squared_error(y_test, predicted)
r2 = r2_score(y_test, predicted)

print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2 ): {r2}")

### END SOLUTION

Mean Squared Error (MSE): 235.07051606583013
R-squared (R2 ): 0.2568698906085112


In [218]:
# Call the `score` method on the model to show the r2 score

### BEGIN SOLUTION
model.score(X_test, y_test)
### END SOLUTION

0.2568698906085112

In [193]:
sorted(zip(model.feature_importances_, feature_names), reverse=True)

[(0.26157979250697866, 'clinics_per_capita'),
 (0.2309006436898862, 'title_10_clients_per_capita'),
 (0.16342869214064576, 'health_dept_per_capita'),
 (0.07922614293602528, 'dept_clinic_title10_per_capita'),
 (0.0729406618311859, 'title10_per_capita'),
 (0.058144106124316246, 'percent_clients_at_pp'),
 (0.04902650965333184, 'pp_clients_per_capita'),
 (0.03127194270702877, 'percent_clinics_tt_funded'),
 (0.03081452416197629, 'pp_per_capita'),
 (0.01662843114386375, 'hospitals_per_capita'),
 (0.004893244877183752, 'binary_pp_tt'),
 (0.0011453082275776302, 'binary_pp')]

## 

In [ ]:
#R2 with raw state data = 0.26
#R2 with per capita = 0.15
#r2 pp, title10, total clinics, states raw plus per capita = .29, pp per capita highest feature importance
#r2 states with per capita and percent of different clinic tt funded, .316